# Initial EDA

In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import json
import gzip
import os
import random
from collections import defaultdict

random.seed(42) # seeding for rerunning purposes

In [236]:
# Monday 
# TODO: scale process to get review data for 1000 books for each pre defined genre
# TODO: Initial Clusters 

In [5]:
def load_data_interactions(file_name, head = 9999):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        # non performant for all data
        for l in fin:
            d = json.loads(l)
            if d['is_read']:
                count += 1
                data.append(d['book_id'])
            
            # break point > head
            #### NEED BETTER WAY TO GET ALL INFORMATION ####
            if (head is not None) and (count > head):
                break
    return data 


def load_data_reviews(file_name, head = 999999):
    count = 0
    data = []
    with gzip.open(file_name) as fin:

        for l in fin:
            d = json.loads(l)
            count += 1
            
            dic = {}
            dic['book_id'] = d['book_id']
            dic['review_text'] = d['review_text']
            
            data.append(dic)
            
            # break point > head
            if (head is not None) and (count > head):
                break
    return data 

def combine_data(root, what):
    assert type(what) is int, 'MUST BE INT'
    predefined_genres = os.listdir(root)
    
    if what == 1:
        data_agg = []
        for genre in predefined_genres:
            is_read = load_data_interactions(os.path.join(root, genre))
#             selected_books = random.sample(raw_data, 1000) # sample 1000 without replacement, list 
            data_agg.append(is_read)
            
        
        final_book_data = [book for selected_books in data_agg for book in selected_books]
        return pd.DataFrame(final_book_data)
    
    if what == 2: 
        data_agg = []
        for genre in predefined_genres:
            print(genre)
            if genre != '.DS_Store':
                read_data = load_data_read(os.path.join(root, genre))
                data_agg.append(read_data)
            else:
                continue
            
        final_book_data = [book for selected_books in data_agg for book in selected_books]
        return pd.DataFrame(final_book_data)
    
    if what == 3: 
        data_agg = []
        for genre in predefined_genres:
            print(genre)
            if genre != '.DS_Store':
                read_data = load_data_reviews(os.path.join(root, genre))
                data_agg.append(read_data)
            else:
                continue
            
        final_book_data = [book for selected_books in data_agg for book in selected_books]
        return pd.DataFrame(final_book_data)

In [6]:
def load_data_read(file_name, head = 999):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        # non performant for all data
        for l in fin:
            d = json.loads(l)
            if d['book_id'] in read_list:
                count += 1
                data.append(d)
            
            # break point > head
            #### NEED BETTER WAY TO GET ALL INFORMATION ####
            if (head is not None) and (count > head):
                break
    return data 

In [7]:
inter = r'/Users/michaelfronda/Desktop/Main/OMSA/22-FA-ISYE6740/data-interactions'
is_read = combine_data(inter, what=1) 

In [8]:
read_list = list(is_read[0])

In [9]:
graphs = r'/Users/michaelfronda/Desktop/Main/OMSA/22-FA-ISYE6740/data-book-graphs'
book_data = combine_data(graphs, what=2)
# LONG RUN 

goodreads_books_comics_graphic.json.gz
goodreads_books_children.json.gz
.DS_Store
goodreads_books_fantasy_paranormal.json.gz
goodreads_books_young_adult.json.gz
goodreads_books_history_biography.json.gz
goodreads_books_romance.json.gz
goodreads_books_mystery_thriller_crime.json.gz
goodreads_books_poetry.json.gz


In [10]:
revs = r'/Users/michaelfronda/Desktop/Main/OMSA/22-FA-ISYE6740/data-reviews'
reviews = combine_data(revs, what=3)

goodreads_reviews_mystery_thriller_crime.json.gz
goodreads_reviews_poetry.json.gz
goodreads_reviews_children.json.gz
goodreads_reviews_romance.json.gz
goodreads_reviews_history_biography.json.gz
goodreads_reviews_fantasy_paranormal.json.gz
goodreads_reviews_young_adult.json.gz
goodreads_reviews_comics_graphic.json.gz


In [342]:
reviews

,book_id,review_text
0,6392944,I haven't read a fun mystery book in a while a...
1,28684704,"A fun, fast paced science fiction thriller. I ..."
2,32283133,http://www.telegraph.co.uk/culture/10...
3,17860739,An amazing and unique creation: JJ Abrams and ...
4,8694005,The Name of the Rose is a thrilling Dan Brown-...
...,...,...
6431528,20176586,Bumping this up to a three star rating primari...
6431529,11465282,The story didn't pull me in although I'm aware...
6431530,31094296,Stunning follow up. Once again it's not the ar...
6431531,27404461,I grew up with the Terran Trade Authority book...


In [11]:
combined = pd.merge(book_data, reviews, how='left', on=['book_id'])

In [12]:
combined['book_id_str'] = combined['book_id'].map(str)
combined['review_text'] = combined['review_text'].map(str)

In [13]:
combined['combined_review'] = combined[['book_id','review_text']].groupby(['book_id'])['review_text'].transform(lambda x: '\t'.join(x))

In [17]:
concatenated_review = combined[['book_id','combined_review']].drop_duplicates()
# LONG RUN 

In [18]:
final_df = pd.merge(book_data, concatenated_review, how='inner', on=['book_id'])

In [19]:
final_df

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series,combined_review
0,0785165037,71,"[520884, 678696]",US,eng,"[{'count': '211', 'name': 'to-read'}, {'count'...",,false,4.10,B00PSN2IHM,...,,2013,https://www.goodreads.com/book/show/17277795-u...,https://images.gr-assets.com/books/1377214930m...,17277795,651,23891166,"Ultimate Comics: Spider-Man, by Brian Michael ...","Ultimate Comics: Spider-Man, by Brian Michael ...",Why does everything always have to be so damn ...
1,1595825010,192,[],US,en-US,"[{'count': '1433', 'name': 'to-read'}, {'count...",,false,4.03,B00A820UFU,...,First Edition,2010,https://www.goodreads.com/book/show/7961571-th...,https://images.gr-assets.com/books/1403206833m...,7961571,2663,9929356,The Amazing Screw-on Head and Other Curious Ob...,The Amazing Screw-on Head and Other Curious Ob...,Stories in the Mignola style and visuals. As a...
2,,1,"[194208, 1114845, 1124201]",US,eng,"[{'count': '322', 'name': 'to-read'}, {'count'...",,false,3.88,,...,ISBN 1563898012 alternate cover,2002,https://www.goodreads.com/book/show/13423869-b...,https://s.gr-assets.com/assets/nophoto/book/11...,13423869,2,103212,Batman/Huntress: Cry for Blood,Batman/Huntress: Cry for Blood,"A retelling of her origin, slightly updated fr..."
3,0785128816,51,[397476],US,eng,"[{'count': '228', 'name': 'to-read'}, {'count'...",,false,3.30,B00AAJR090,...,Premiere,2008,https://www.goodreads.com/book/show/5626060-hu...,https://s.gr-assets.com/assets/nophoto/book/11...,5626060,602,5797459,"Hulk, Volume 1: Red Hulk","Hulk, Volume 1: Red Hulk",Loeb takes a great mystery and turns it in a l...
4,0316358452,125,[199703],US,eng,"[{'count': '407', 'name': 'comics'}, {'count':...",,false,4.17,,...,American Edition,1976,https://www.goodreads.com/book/show/179172.Des...,https://images.gr-assets.com/books/1344265829m...,179172,7997,2278369,"Destination Moon (Tintin, #16)","Destination Moon (Tintin, #16)",The first part of Herge's two part 'Trip to th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0571176550,99,[],US,eng,"[{'count': '2480', 'name': 'to-read'}, {'count...",,false,4.15,,...,Faber Library Edition,1996,https://www.goodreads.com/book/show/97553.Crow,https://images.gr-assets.com/books/1468984529m...,97553,1983,94172,Crow: From the Life and Songs of the Crow,Crow: From the Life and Songs of the Crow,"What a horrible lil' dreamscape. Seriously, th..."
7996,,51,[],US,ara,"[{'count': '264', 'name': 'to-read'}, {'count'...",,true,3.40,,...,manuscript,2014,https://www.goodreads.com/book/show/20878228,https://images.gr-assets.com/books/1398243693m...,20878228,146,40218004,الحب جالس في مقهى الماضي,الحب جالس في مقهى الماضي,akhr m Sdr l swzn `lywn \n jmyl@ k `dth .. wn ...
7997,0003027600,49,[],US,per,"[{'count': '349', 'name': 'to-read'}, {'count'...",,false,4.55,,...,,,https://www.goodreads.com/book/show/230246._,https://s.gr-assets.com/assets/nophoto/book/11...,230246,1394,222980,دیوان كلیات شمس تبریزی,دیوان كلیات شمس تبریزی,"very good\tBrilliant!if you want to know how ""..."
7998,0385244843,5,[],US,eng,"[{'count': '546', 'name': 'to-read'}, {'count'...",,false,4.31,,...,,1989,https://www.goodreads.com/book/show/286114.Hai...,https://images.gr-assets.com/books/1320507426m...,286114,31,11442,Hailstones and Halibut Bones,Hailstones and Halibut Bones,I purchased this for the light and colors unit...


In [325]:
d = defaultdict(str)

In [331]:
for n in ['apple', 'orange', 'grape']:
    d['a'] += n

In [332]:
dict(d)

{'a': 'grapeappleorangegrape'}

In [319]:
d = {'a':1, 'b':'apple'}

lis = []

lis.append([d['a'],d['b']])
lis.append([d['a'],d['b']])
pd.DataFrame(lis)

,0,1
0,1,apple
1,1,apple


In [320]:
d.keys

<function dict.keys>

In [238]:
# Tuesday 
# TODO: Continue Clustering books, different methods?

In [54]:
raw_data.sample(10)

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
3556,1467712345,27,[],US,,"[{'count': '38', 'name': 'to-read'}, {'count':...",,false,3.26,B00SG65DMU,...,4,,2015,https://www.goodreads.com/book/show/19354685-t...,https://images.gr-assets.com/books/1416180554m...,19354685,81,27415839,Trash Mountain,Trash Mountain
4727,0340917555,53,[146020],US,en-GB,"[{'count': '1012', 'name': 'to-read'}, {'count...",,false,3.78,B007LO1KVQ,...,,,2006,https://www.goodreads.com/book/show/124675.The...,https://images.gr-assets.com/books/1415582629m...,124675,3205,120063,"The Secret Seven Adventure (The Secret Seven, #2)","The Secret Seven Adventure (The Secret Seven, #2)"
335,0750013931,3,[381005],US,,"[{'count': '224', 'name': 'to-read'}, {'count'...",,false,4.23,,...,,,,https://www.goodreads.com/book/show/1914630.Th...,https://s.gr-assets.com/assets/nophoto/book/11...,1914630,41,80619,"The Oaken Throne (The Deptford Histories, #2)","The Oaken Throne (The Deptford Histories, #2)"
584,0823424006,25,[],US,,"[{'count': '23', 'name': 'to-read'}, {'count':...",,false,3.47,,...,2,,2012,https://www.goodreads.com/book/show/12710065-f...,https://images.gr-assets.com/books/1328019578m...,12710065,87,17843561,Fox Tails: Four Fables from Aesop,Fox Tails: Four Fables from Aesop
8990,9953875367,3,[],US,ara,"[{'count': '2505', 'name': 'to-read'}, {'count...",,false,3.90,,...,,,2008,https://www.goodreads.com/book/show/8082219,https://s.gr-assets.com/assets/nophoto/book/11...,8082219,6,1021311,عفريت الأرقام,عفريت الأرقام
9120,1612440916,6,[],US,en-US,"[{'count': '187', 'name': 'to-read'}, {'count'...",,false,4.17,B008AK7ALE,...,8,,2012,https://www.goodreads.com/book/show/15831747-v...,https://images.gr-assets.com/books/1367002342m...,15831747,13,21373662,Vin and the Dorky Duet,Vin and the Dorky Duet
8751,,3,[155079],US,eng,"[{'count': '89', 'name': 'to-read'}, {'count':...",,false,3.96,B00GVFU7R4,...,,,2009,https://www.goodreads.com/book/show/1137256.Ca...,https://images.gr-assets.com/books/1338051257m...,1137256,14,1124501,"Caddy Ever After (Casson Family, #4)","Caddy Ever After (Casson Family, #4)"
5684,8877828226,2,[206569],US,ita,"[{'count': '216', 'name': 'to-read'}, {'count'...",,false,3.95,,...,1,Brutte Storie,1999,https://www.goodreads.com/book/show/9749359-i-...,https://images.gr-assets.com/books/1445556344m...,9749359,15,413020,I cinici celti,I cinici celti
5131,,1,[],US,,"[{'count': '568', 'name': 'to-read'}, {'count'...",B00JMV8TAG,true,4.05,B00JMV8TAG,...,,,,https://www.goodreads.com/book/show/22930358-a...,https://s.gr-assets.com/assets/nophoto/book/11...,22930358,13,3719102,AN OLD-FASHIONED GIRL. (Annotated) (Louisa May...,AN OLD-FASHIONED GIRL. (Annotated) (Louisa May...
6135,,1,[],US,,"[{'count': '976', 'name': 'to-read'}, {'count'...",,false,3.89,,...,,,,https://www.goodreads.com/book/show/26217381-g...,https://images.gr-assets.com/books/1441263911m...,26217381,23,2801485,Gümüş Patenler,Gümüş Patenler


In [9]:
raw_data.columns

Index(['isbn', 'text_reviews_count', 'series', 'country_code', 'language_code',
       'popular_shelves', 'asin', 'is_ebook', 'average_rating', 'kindle_asin',
       'similar_books', 'description', 'format', 'link', 'authors',
       'publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'edition_information', 'publication_year', 'url',
       'image_url', 'book_id', 'ratings_count', 'work_id', 'title',
       'title_without_series'],
      dtype='object')

In [33]:
# popular shelves - top user generated shelves for a book, defines genres 
raw_data['popular_shelves'][96]

[{'count': '7173', 'name': 'to-read'},
 {'count': '235', 'name': 'vampires'},
 {'count': '195', 'name': 'currently-reading'},
 {'count': '194', 'name': 'young-adult'},
 {'count': '186', 'name': 'favorites'},
 {'count': '109', 'name': 'fantasy'},
 {'count': '104', 'name': 'vampire'},
 {'count': '81', 'name': 'owned'},
 {'count': '80', 'name': 'books-i-own'},
 {'count': '77', 'name': 'ya'},
 {'count': '76', 'name': 'series'},
 {'count': '73', 'name': 'paranormal'},
 {'count': '50', 'name': 'supernatural'},
 {'count': '42', 'name': 'heather-brewer'},
 {'count': '41', 'name': 'fiction'},
 {'count': '37', 'name': 'horror'},
 {'count': '31', 'name': 'teen'},
 {'count': '31', 'name': 'urban-fantasy'},
 {'count': '30', 'name': 'the-chronicles-of-vladimir-tod'},
 {'count': '30', 'name': 'vladimir-tod'},
 {'count': '21', 'name': 'romance'},
 {'count': '21', 'name': 'library'},
 {'count': '18', 'name': 'owned-books'},
 {'count': '18', 'name': 'to-buy'},
 {'count': '16', 'name': 'high-school'},
 {

In [32]:
display(raw_data['similar_books'][96]) # list of books that users who like the current book also like

display(raw_data.iloc[96,:])

['25861113',
 '7430195',
 '18765937',
 '6120544',
 '3247550',
 '9266753',
 '6976966',
 '25764778',
 '17696585',
 '23256849',
 '642463',
 '2842796',
 '263172',
 '10874337']

isbn                                                                     
text_reviews_count                                                      9
series                                                           [176160]
country_code                                                           US
language_code                                                         eng
popular_shelves         [{'count': '7173', 'name': 'to-read'}, {'count...
asin                                                           B0042JSOQC
is_ebook                                                             true
average_rating                                                       4.35
kindle_asin                                                    B004IYJDXY
similar_books           [25861113, 7430195, 18765937, 6120544, 3247550...
description             It all comes down to this.\nVlad's running out...
format                                                                   
link                    https://www.go

### Checks for 'similar_books' 

Are they book ids? or ISBNs?

In [25]:
raw_data[raw_data['book_id'] == 7430195]

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series


In [40]:
mask = raw_data.isbn.apply(lambda x: '25861113' == x) # 
raw_data[mask] 

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series


In [41]:
# check entire dataset for similar books